# STAT207 Final Project - INSERT TITLE HERE


YOUR NAMES

In [58]:
#Imports here
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import statsmodels.formula.api as smf

# sklearn Linear
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.metrics import r2_score

# sklearn Logistic
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# Ignore scikit-learn ConvergenceWarnings
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [59]:
def plot_roc(fpr, tpr, auc, lw=2):
    plt.plot(fpr, tpr, color="darkorange",lw=lw,label="ROC curve (area = "+str(round(auc,3))+")")
    plt.plot([0,1], [0,1], color="navy",lw=lw, linestyle="--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend(loc="lower right")
    plt.show()

def fpr_tpr_thresh(y, pred_prob, thresh):
    yhat = 1 * (pred_prob >= thresh)
    tn, fp, fn, tp = confusion_matrix(y_true=y, y_pred=yhat).ravel()
    tpr = tp / (fn + tp)
    fpr = fp / (fp + tn)
    return {"thresh": thresh, "fpr": fpr, "tpr": tpr}

## 1. Introduction

## 2. Dataset Discussion

In [60]:
df = pd.read_csv("coffee.csv")
df.head()

,Location.Country,Location.Region,Location.Altitude.Min,Location.Altitude.Max,Location.Altitude.Average,Year,Data.Owner,Data.Type.Species,Data.Type.Variety,Data.Type.Processing method,...,Data.Scores.Flavor,Data.Scores.Aftertaste,Data.Scores.Acidity,Data.Scores.Body,Data.Scores.Balance,Data.Scores.Uniformity,Data.Scores.Sweetness,Data.Scores.Moisture,Data.Scores.Total,Data.Color
0,United States,kona,0,0,0,2010,kona pacific farmers cooperative,Arabica,NaN,NaN,...,8.42,8.08,7.75,7.67,7.83,10.0,10.0,0.00,86.25,Unknown
1,Brazil,sul de minas - carmo de minas,12,12,12,2010,jacques pereira carneiro,Arabica,Yellow Bourbon,NaN,...,7.92,7.92,7.75,8.33,8.00,10.0,10.0,0.08,86.17,Unknown
2,Brazil,sul de minas - carmo de minas,12,12,12,2010,jacques pereira carneiro,Arabica,Yellow Bourbon,NaN,...,7.92,8.00,7.75,7.92,8.00,10.0,10.0,0.01,86.17,Unknown
3,Ethiopia,sidamo,0,0,0,2010,ethiopia commodity exchange,Arabica,NaN,NaN,...,8.00,7.83,8.00,7.92,7.83,10.0,10.0,0.00,85.08,Unknown
4,Ethiopia,sidamo,0,0,0,2010,ethiopia commodity exchange,Arabica,NaN,NaN,...,7.83,7.58,8.00,7.83,7.50,10.0,10.0,0.10,83.83,Unknown


## Cleaning

### Creating Two-Level Response Variable

In [61]:
df["Data.Type.Variety"].value_counts()


Data.Type.Variety
Caturra                  205
Bourbon                  179
Typica                   135
Other                    101
Catuai                    63
Hawaiian Kona             44
Mundo Novo                25
SL14                      17
Catimor                   16
SL28                      13
Gesha                     12
Pacas                     11
Yellow Bourbon             8
SL34                       7
Pacamara                   6
Arusha                     6
Mandheling                 3
Sumatra                    2
Ethiopian Yirgacheffe      2
Java                       2
Marigojipe                 1
Sumatra Lintong            1
Peaberry                   1
Blue Mountain              1
Pache Comun                1
Ruiru 11                   1
Sulawesi                   1
Name: count, dtype: int64

In [62]:
df = df[df["Data.Type.Variety"].isin(["Caturra", "Bourbon"])]
df

,Location.Country,Location.Region,Location.Altitude.Min,Location.Altitude.Max,Location.Altitude.Average,Year,Data.Owner,Data.Type.Species,Data.Type.Variety,Data.Type.Processing method,...,Data.Scores.Flavor,Data.Scores.Aftertaste,Data.Scores.Acidity,Data.Scores.Body,Data.Scores.Balance,Data.Scores.Uniformity,Data.Scores.Sweetness,Data.Scores.Moisture,Data.Scores.Total,Data.Color
10,Brazil,south of minas,1250,1250,1250,2011,nucoffee,Arabica,Bourbon,Natural / Dry,...,8.50,8.00,8.00,8.00,8.00,10.00,10.00,0.12,86.92,Green
11,Brazil,vale da grama,1300,1300,1300,2011,nucoffee,Arabica,Bourbon,Natural / Dry,...,8.17,8.00,7.75,8.00,8.00,10.00,10.00,0.12,86.42,Green
12,Brazil,south of minas,1250,1250,1250,2011,nucoffee,Arabica,Bourbon,Semi-washed / Semi-pulped,...,8.00,8.00,8.00,7.75,8.00,10.00,10.00,0.11,86.08,Blue-Green
13,Brazil,south of minas,1250,1250,1250,2011,nucoffee,Arabica,Bourbon,Semi-washed / Semi-pulped,...,8.00,8.00,7.67,7.75,8.00,10.00,10.00,0.11,85.42,Blue-Green
14,Brazil,south of minas,1200,1200,1200,2011,nucoffee,Arabica,Bourbon,Natural / Dry,...,7.92,7.75,7.75,8.00,7.75,10.00,10.00,0.11,84.92,Green
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,Nicaragua,jinotega,1250,1250,1250,2017,"comercial internacional exportadora, s.a.",Arabica,Caturra,Washed / Wet,...,7.08,6.92,7.08,7.42,7.17,10.00,10.00,0.10,80.00,Green
981,Guatemala,san marcos,1700,1700,1700,2017,juan luis alvarado romero,Arabica,Bourbon,Washed / Wet,...,7.25,6.92,7.33,7.25,7.33,9.33,10.00,0.10,79.83,Green
982,Guatemala,santa rosa,1219,1219,1219,2017,juan luis alvarado romero,Arabica,Bourbon,Washed / Wet,...,7.42,7.33,7.42,7.50,7.42,8.67,9.33,0.10,79.17,Green
984,Guatemala,san marcos,1700,1700,1700,2017,juan luis alvarado romero,Arabica,Bourbon,Washed / Wet,...,7.67,7.42,7.42,7.67,7.67,8.67,8.67,0.10,79.08,Green


### Isolating and Renaming Relevant Columns

In [63]:
df = df[["Location.Country", "Data.Type.Variety", "Data.Color", "Data.Scores.Flavor", "Data.Scores.Aftertaste", "Data.Scores.Acidity", "Data.Scores.Body", "Data.Scores.Balance", "Data.Scores.Uniformity", "Data.Scores.Sweetness", "Data.Scores.Moisture"]]
df.rename(columns={"Location.Country": "country", "Data.Type.Variety": "variety", "Data.Scores.Flavor": "flavor", "Data.Scores.Aftertaste": "aftertaste", "Data.Scores.Acidity": "acidity", "Data.Scores.Body": "body", "Data.Scores.Balance": "balance", "Data.Scores.Uniformity": "uniformity", "Data.Scores.Sweetness": "sweetness", "Data.Scores.Moisture": "moisture", "Data.Color": "color"}, inplace=True)
df

/var/folders/41/_gkgvhb94wd4156zplzr4cg00000gn/T/ipykernel_60054/3116509593.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"Location.Country": "country", "Data.Type.Variety": "variety", "Data.Scores.Flavor": "flavor", "Data.Scores.Aftertaste": "aftertaste", "Data.Scores.Acidity": "acidity", "Data.Scores.Body": "body", "Data.Scores.Balance": "balance", "Data.Scores.Uniformity": "uniformity", "Data.Scores.Sweetness": "sweetness", "Data.Scores.Moisture": "moisture", "Data.Color": "color"}, inplace=True)


,country,variety,color,flavor,aftertaste,acidity,body,balance,uniformity,sweetness,moisture
10,Brazil,Bourbon,Green,8.50,8.00,8.00,8.00,8.00,10.00,10.00,0.12
11,Brazil,Bourbon,Green,8.17,8.00,7.75,8.00,8.00,10.00,10.00,0.12
12,Brazil,Bourbon,Blue-Green,8.00,8.00,8.00,7.75,8.00,10.00,10.00,0.11
13,Brazil,Bourbon,Blue-Green,8.00,8.00,7.67,7.75,8.00,10.00,10.00,0.11
14,Brazil,Bourbon,Green,7.92,7.75,7.75,8.00,7.75,10.00,10.00,0.11
...,...,...,...,...,...,...,...,...,...,...,...
979,Nicaragua,Caturra,Green,7.08,6.92,7.08,7.42,7.17,10.00,10.00,0.10
981,Guatemala,Bourbon,Green,7.25,6.92,7.33,7.25,7.33,9.33,10.00,0.10
982,Guatemala,Bourbon,Green,7.42,7.33,7.42,7.50,7.42,8.67,9.33,0.10
984,Guatemala,Bourbon,Green,7.67,7.42,7.42,7.67,7.67,8.67,8.67,0.10


### Sample Size Cleaning

In [64]:
top_countries = df["country"].value_counts()
top_countries

country
Guatemala                       141
Colombia                        105
Mexico                           46
Costa Rica                       24
Honduras                         18
Brazil                           15
Nicaragua                         9
Tanzania, United Republic Of      7
El Salvador                       4
Uganda                            3
Panama                            2
Vietnam                           2
Malawi                            2
Burundi                           1
Thailand                          1
Peru                              1
Rwanda                            1
Ethiopia                          1
Indonesia                         1
Name: count, dtype: int64

In [65]:
kept_countries = top_countries[top_countries > 10].index
df_top = df[df["country"].isin(kept_countries)]
df_top

,country,variety,color,flavor,aftertaste,acidity,body,balance,uniformity,sweetness,moisture
10,Brazil,Bourbon,Green,8.50,8.00,8.00,8.00,8.00,10.00,10.00,0.12
11,Brazil,Bourbon,Green,8.17,8.00,7.75,8.00,8.00,10.00,10.00,0.12
12,Brazil,Bourbon,Blue-Green,8.00,8.00,8.00,7.75,8.00,10.00,10.00,0.11
13,Brazil,Bourbon,Blue-Green,8.00,8.00,7.67,7.75,8.00,10.00,10.00,0.11
14,Brazil,Bourbon,Green,7.92,7.75,7.75,8.00,7.75,10.00,10.00,0.11
...,...,...,...,...,...,...,...,...,...,...,...
976,Guatemala,Caturra,Green,7.25,7.08,7.17,7.25,7.17,10.00,10.00,0.10
981,Guatemala,Bourbon,Green,7.25,6.92,7.33,7.25,7.33,9.33,10.00,0.10
982,Guatemala,Bourbon,Green,7.42,7.33,7.42,7.50,7.42,8.67,9.33,0.10
984,Guatemala,Bourbon,Green,7.67,7.42,7.42,7.67,7.67,8.67,8.67,0.10


### Selected Variables
Our chosen response variable is `variety`. This variable represents the variety of coffee of each row, and has two distince values, `Bourbon`, and `Caturra`. We are using 2 categorical explanatory variables and 8 numerical explanatory variabels. The categorical explanatory variables are `color`, and `country`, the color of the coffee plant, and the country where the plant was produced, respectively. The 8 numerical explanatory variables are scores on a 1-10 scale of various properties of the coffee, which are `flavor`, `aftertaste`, `acidity`, `body`, `balance`, `uniformity`, `sweetness`, and `moisture`. We chose to focus on these explanatory variables because...

## 4. Preliminary Analysis

## 5. Model Data Preprocessing

## 6. Feature Selection with k-Fold Cross-Validation

## 7. Best Model Discussion

## 8. Additional Analysis/Insight

In [ ]:
## 1. Introduction
Our primary research goal is to build a predictive model that will effectively predict `variety` of a coffee in new datasets. Our secondary research goal is that ideally we would like our chosen model to yield reliable interpretable insights about the nature of the relationship between the variables in the dataset. The motivation for building a model like this is to be able to classify the variety of unknown coffee strains. The response variable `variety` has two levels, *Caturra*, and *Bourbon*. According to counterculturecoffee.com, "Caturra is a genetic mutation of Bourbon, first identified outside of a village called Caturra in Brazil. This variety was selected and cultivated for its smaller plant size and higher level of production compared to Bourbon."$^1$. Neither of them lend themselves to being the "positive outcome", so equally high accuracy for both *Caturra* and *Bourbon* is desirable. According to World Coffee Research, "The land available to grow Arabica coffee is also expected to dramatically shrink in coming years, meaning coffee trees must become more productive to meet increasing demand. Without better planting material, the coffee industry faces a potentially disastrous decline in Arabica supply in the coming decades."$^2$ Better coffee varieties need to be bred, and a classifier to determine whether a new variety is more similar to Bourbon or Caturra may be useful.





## 1. Introduction
Our primary research goal is to build a predictive model that will effectively predict `variety` of a coffee in new datasets. Our secondary research goal is that ideally we would like our chosen model to yield reliable interpretable insights about the nature of the relationship between the variables in the dataset. The motivation for building a model like this is to be able to classify the variety of unknown coffee strains. The response variable `variety` has two levels, *Caturra*, and *Bourbon*. According to counterculturecoffee.com, "Caturra is a genetic mutation of Bourbon, first identified outside of a village called Caturra in Brazil. This variety was selected and cultivated for its smaller plant size and higher level of production compared to Bourbon."$^1$. Neither of them lend themselves to being the "positive outcome", so equally high accuracy for both *Caturra* and *Bourbon* is desirable. According to World Coffee Research, "The land available to grow Arabica coffee is also expected to dramatically shrink in coming years, meaning coffee trees must become more productive to meet increasing demand. Without better planting material, the coffee industry faces a potentially disastrous decline in Arabica supply in the coming decades."$^2$ Better coffee varieties need to be bred, and a classifier to determine whether a new variety is more similar to Bourbon or Caturra may be useful.





## 9. Conclusion

## References